In [39]:
import pandas as pd
from pandas.api.types import CategoricalDtype
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pymysql
# from sqlalchemy import create_engine
# !pip install scikit-learn
from sklearn.preprocessing import MinMaxScaler
from scipy.stats import skew, kurtosis
import os

In [40]:
# matplotlib 한글 사용

plt.rcParams["font.family"] ="Malgun Gothic"
plt.rcParams["axes.unicode_minus"] =False

# 1. 데이터 불러와서 가공하기

In [41]:
# 미세먼지 데이터와 날씨 데이터 불러와서 각각 데이터프레임에 저장
fine_dust_df = pd.read_csv("./data/fine_dust.csv")
weather_df = pd.read_csv("./data/weather_data.csv")
holiday_df = pd.read_csv("./data/hd_21_24.csv", encoding = "CP949")

# 미세먼지 데이터프레임과 날씨 데이터프레임의 컬럼 조회
fine_dust_df.columns, weather_df.columns, holiday_df.columns

(Index(['Unnamed: 0', 'station_id', 'station_name', 'date', 'fine_dust(㎍/㎥)'], dtype='object'),
 Index(['Unnamed: 0', 'station_id', 'station_name', 'date_time', 'avg_temp',
        'min_temp', 'max_temp', 'daily_precipitation', 'avg_wind_speed',
        'daily_snow_depth'],
       dtype='object'),
 Index(['일시', 'dateName'], dtype='object'))

### 1-1. 분석에 용이하게 데이터프레임 가공하기

날씨 데이터

In [42]:
# 필요 컬럼을 원하는 순서로 정렬

fine_dust_df = fine_dust_df[["date", "fine_dust(㎍/㎥)"]]
weather_df = weather_df[["date_time", "avg_temp", "min_temp", "max_temp", 
                         "daily_precipitation", "daily_snow_depth", "avg_wind_speed"]]

In [43]:
# 각 데이터 프레임 정보 확인
fine_dust_df.info(), weather_df.info(), holiday_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1323 entries, 0 to 1322
Data columns (total 2 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   date            1323 non-null   object
 1   fine_dust(㎍/㎥)  1323 non-null   int64 
dtypes: int64(1), object(1)
memory usage: 20.8+ KB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1399 entries, 0 to 1398
Data columns (total 7 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   date_time            1399 non-null   object 
 1   avg_temp             1399 non-null   float64
 2   min_temp             1398 non-null   float64
 3   max_temp             1399 non-null   float64
 4   daily_precipitation  591 non-null    float64
 5   daily_snow_depth     92 non-null     float64
 6   avg_wind_speed       1394 non-null   float64
dtypes: float64(6), object(1)
memory usage: 76.6+ KB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 66 entri

(None, None, None)

In [50]:
# 데이터프레임을 하나의 데이터 프레임으로 합치기

# 각 데이터프레임에서 날짜를 기준으로 합치기위해 컬럼명 수정하기
    # 날씨 데이터프레임에서는 일부 컬럼만 이름 수정
weather_df = weather_df.rename(columns={"date_time":"date", 
                                        "daily_precipitation":"precipitation(mm)", 
                                        "daily_snow_depth":"snow_depth(cm)"})
    # 공휴일 데이터프레임에서는 전체 컬럼 이름 수정
holiday_df.columns=["date", "datename"]

# 미세먼지와 날씨 데이터프레임 합치기
fd_and_wt = pd.merge(left=fine_dust_df, right=weather_df, how="outer", on="date")
# # 판다스의 merge를 활용하여 date컬럼을 기준으로 왼쪽에는 미세먼지 데이터프레임, 오른쪽에는 날씨 데이터프레임을 합침
# # 이 과정에서 how를 "outer"로 지정하여 결측치를 없애지 않고 NaN으로 받아 합침

# 위와 같은 방식으로 합쳐진 날씨 데이터프레임과 공휴일 데이터프레임 합치기
all_df = pd.merge(left=fd_and_wt, right=holiday_df, how="outer", on="date")

# 미세먼지, 날씨, 공휴일 모두 합친 데이터프레임의 정보 확인
all_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1400 entries, 0 to 1399
Data columns (total 9 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   date               1400 non-null   object 
 1   fine_dust(㎍/㎥)     1323 non-null   float64
 2   avg_temp           1399 non-null   float64
 3   min_temp           1398 non-null   float64
 4   max_temp           1399 non-null   float64
 5   precipitation(mm)  591 non-null    float64
 6   snow_depth(cm)     92 non-null     float64
 7   avg_wind_speed     1394 non-null   float64
 8   datename           66 non-null     object 
dtypes: float64(7), object(2)
memory usage: 98.6+ KB


### 1-2. 날짜를 기준으로 년, 계절, 평일/주말(+공휴일) 구분하기

1-2-1. 평일/주말(+공휴일) 구분하기

In [60]:
# 날짜를 기준으로 년, 계절, 평일/주말(+공휴일)을 지정하기 위해 데이터프레임의 date 컬럼의 타입을 object에서 datetime으로 바꾸기
all_df["date"] = pd.to_datetime(all_df["date"])

# date컬럼의 자료형이 변환되었는지 확인하기
all_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1400 entries, 0 to 1399
Data columns (total 10 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   date               1400 non-null   datetime64[ns]
 1   fine_dust(㎍/㎥)     1323 non-null   float64       
 2   avg_temp           1399 non-null   float64       
 3   min_temp           1398 non-null   float64       
 4   max_temp           1399 non-null   float64       
 5   precipitation(mm)  591 non-null    float64       
 6   snow_depth(cm)     92 non-null     float64       
 7   avg_wind_speed     1394 non-null   float64       
 8   datename           66 non-null     object        
 9   day                1400 non-null   object        
dtypes: datetime64[ns](1), float64(7), object(2)
memory usage: 109.5+ KB


In [61]:
# 파이썬은 datetime 자료형에서 dt.weekday를 사용하여 0~4는 월~금으로, 5, 6은 토, 일로 값을 메겨 평일/주말 여부를 판단 할 수 있다.
# 따라서 np.where를 사용하여서 요일 숫자 값이 5미만이면 평일(Weekday)로, 아니라면 주말(Weekend)로 값을 지정할 수 있다.
# 데이터프레임에 "day"라는 컬럼을 만들어 지정된 값을 넣을 수 있다.
all_df["day"] = np.where(all_df["date"].dt.weekday < 5, "Weekday", "Weekend")

# day컬럼이 잘 들어갔는지 확인
all_df.head(2)

,date,fine_dust(㎍/㎥),avg_temp,min_temp,max_temp,precipitation(mm),snow_depth(cm),avg_wind_speed,datename,day
0,2021-01-01,36.0,-4.2,-9.8,1.6,NaN,NaN,2.0,1월1일,Weekday
1,2021-01-02,43.0,-5.0,-8.4,-1.4,NaN,NaN,2.6,NaN,Weekend


In [ ]:
# dt.weekday를 사용하여 평일/주말을 나눴으나, 공휴일 정보가 없어 1월 1일(공휴일)이 평일(Weekday)로 표시된다.
# 이 문제를 해결하기위해 공휴일 데이터프레임(holiday_df)을 사용한다.

In [66]:
# 공휴일은 주말과 같은 값(Weekend) 주기
# date_name2 시리즈를 만들어서 datename이 있다면, 공휴일이므로 주말로 처리한다 -> 공휴일은 Weekend로 값을 준다
# datename에 값이 없는 NaN이라면 공휴일이 아니므로, day컬럼에서 평일/주말 값을 가져와서 date_name2 시리즈에 채우기

# apply 활용
date_name2 = all_df.apply(lambda row : 'Weekend' if str(row['datename']) != 'nan' else row['day'], axis=1)
# date_name2시리즈 생성해서 datename 컬럼을 문자열로 바꿔 datename에 값이 있다면(=nan이 아니라면)
# date_name2 시리즈에 Weekend로 값 채우기
# nan 이라면 day컬럼의 값으로 채우기

# 생성한 date_name2시리즈를 all_df에 day_type컬럼을 생성해서 합치기
all_df['day_type'] = date_name2

# 확인하기
all_df.head(2), all_df.tail(2)

(        date  fine_dust(㎍/㎥)  avg_temp  min_temp  max_temp  precipitation(mm)  \
 0 2021-01-01            36.0      -4.2      -9.8       1.6                NaN   
 1 2021-01-02            43.0      -5.0      -8.4      -1.4                NaN   
 
    snow_depth(cm)  avg_wind_speed datename      day day_type  
 0             NaN             2.0     1월1일  Weekday  Weekend  
 1             NaN             2.6      NaN  Weekend  Weekend  ,
            date  fine_dust(㎍/㎥)  avg_temp  min_temp  max_temp  \
 1398 2024-10-30            28.0      15.7      11.4      21.4   
 1399 2024-12-25             NaN       NaN       NaN       NaN   
 
       precipitation(mm)  snow_depth(cm)  avg_wind_speed datename      day  \
 1398                NaN             NaN             1.6      NaN  Weekday   
 1399                NaN             NaN             NaN    기독탄신일  Weekday   
 
      day_type  
 1398  Weekday  
 1399  Weekend  )

In [77]:
# 필요한 컬럼과 행만 추출하기
all_df = all_df.loc[:1398, ['date', 'fine_dust(㎍/㎥)', 'avg_temp', 'min_temp', 'max_temp', 
                            'precipitation(mm)', 'snow_depth(cm)', 'avg_wind_speed', 'day_type']]

# 필요한 컬럼과 행만 추출되었는지 확인하기
all_df.head(2), all_df.tail(2)

(        date  fine_dust(㎍/㎥)  avg_temp  min_temp  max_temp  precipitation(mm)  \
 0 2021-01-01            36.0      -4.2      -9.8       1.6                NaN   
 1 2021-01-02            43.0      -5.0      -8.4      -1.4                NaN   
 
    snow_depth(cm)  avg_wind_speed day_type  
 0             NaN             2.0  Weekend  
 1             NaN             2.6  Weekend  ,
            date  fine_dust(㎍/㎥)  avg_temp  min_temp  max_temp  \
 1397 2024-10-29             NaN      15.0      13.4      18.3   
 1398 2024-10-30            28.0      15.7      11.4      21.4   
 
       precipitation(mm)  snow_depth(cm)  avg_wind_speed day_type  
 1397                0.0             NaN             2.3  Weekday  
 1398                NaN             NaN             1.6  Weekday  )

1-2-2. 연도별, 계절별로 구분하기

In [80]:
# date를 기준으로 계절별 구분 
# (봄: 3~5, 여름: 6~8, 가을: 9~11, 겨울: 12~2)
conditions = [
    ( all_df["date"].dt.month >= 3 ) & ( all_df["date"].dt.month <= 5 ),
    ( all_df["date"].dt.month >= 6 ) & ( all_df["date"].dt.month <= 8 ),
    ( all_df["date"].dt.month >= 9 ) & ( all_df["date"].dt.month <= 11 ),
    ( all_df["date"].dt.month <= 2 ) | ( all_df["date"].dt.month >= 12 )
]
Season = ["Spring", "Summer", "Fall", "Winter"]
all_df["season"] = np.select(conditions, Season, default="unknown")

print( all_df.iloc[58:60, :] )
print( all_df.iloc[150:152, :] )
print( all_df.iloc[242:244, :] )
print( all_df.iloc[333:335, :] )

         date  fine_dust(㎍/㎥)  avg_temp  min_temp  max_temp  \
58 2021-02-28            32.0       7.8       3.2      10.8   
59 2021-03-01             NaN       4.7       0.5       9.0   

    precipitation(mm)  snow_depth(cm)  avg_wind_speed day_type  season  
58                NaN             NaN             2.1  Weekend  Winter  
59               67.5             2.3             3.5  Weekend  Spring  
          date  fine_dust(㎍/㎥)  avg_temp  min_temp  max_temp  \
150 2021-05-31            59.0      19.9      16.5      23.8   
151 2021-06-01            52.0      20.2      15.9      23.9   

     precipitation(mm)  snow_depth(cm)  avg_wind_speed day_type  season  
150                4.0             NaN             1.8  Weekday  Spring  
151                3.2             NaN             2.2  Weekday  Summer  
          date  fine_dust(㎍/㎥)  avg_temp  min_temp  max_temp  \
242 2021-08-31            37.0      20.5      18.1      23.1   
243 2021-09-01             9.0      21.4      18